In [ ]:
!pip3 install datasets


In [1]:
from datasets import load_dataset
from PIL import Image
from io import BytesIO
import requests
import os
import json
import uuid
import numpy as np
import skimage
import math

In [2]:
class Util:

  @staticmethod
  def parameter(start, end, delta=0):

    value = np.random.randint(start-delta, end+delta)
    parameters = len(range(start-delta, end+delta))

    return value, parameters

In [3]:
DELTA_MIN = 20
DELTA_MAX = 80
SIZE = (100, 100)

In [4]:
def angle(flags=[False,False], preset=None):


    var_y = flags[0]
    var_x = flags[1]


    sparse = None
    image = None
    label = None
    parameters = 1


    Y_RANGE = (DELTA_MIN, DELTA_MAX)
    X_RANGE = (DELTA_MIN, DELTA_MAX)

    DOF = 90
    ANGLE = np.random.randint(1, DOF+1)
    parameters *= DOF

    if preset:
      ANGLE=preset

    LENGTH = DELTA_MIN

    X = int(SIZE[1] / 2)
    if var_x:
      X, p = Util.parameter(X_RANGE[0], X_RANGE[1])
      parameters *= p

    Y = int(SIZE[0] / 2)
    if var_y:
      Y, p = Util.parameter(Y_RANGE[0], Y_RANGE[1])
      parameters *= p

    image = np.zeros(SIZE, dtype=bool)

    # first line
    first_angle = np.random.randint(360)
    theta = -(np.pi / 180.0) * first_angle
    END = (Y - LENGTH * np.cos(theta), X - LENGTH * np.sin(theta))
    rr, cc = skimage.draw.line(Y, X, int(np.round(END[0])), int(np.round(END[1])))
    image[rr, cc] = 1

    second_angle = first_angle+ANGLE
    theta = -(np.pi / 180.0) * second_angle
    END = (Y - LENGTH * np.cos(theta), X - LENGTH * np.sin(theta))
    rr, cc = skimage.draw.line(Y, X, int(np.round(END[0])), int(np.round(END[1])))
    image[rr, cc] = 1

    sparse = [Y, X, ANGLE, first_angle]

    label = ANGLE

    return sparse, image, label, parameters

In [5]:
def length(flags=[False, False, False], preset=None):

    var_y = flags[0]
    var_x = flags[1]
    var_width = flags[2]


    sparse = None
    image = None
    label = None
    parameters = 1

    Y_RANGE = (DELTA_MIN, DELTA_MAX)
    X_RANGE = (DELTA_MIN, DELTA_MAX)

    LENGTH, p = Util.parameter(1, Y_RANGE[1]-Y_RANGE[0]+1) # 1..60
    parameters *= p

    if preset:
      LENGTH = preset

    MAX_LENGTH = Y_RANGE[1]-Y_RANGE[0]
    # print 'Max length', MAX_LENGTH

    X = math.floor(SIZE[1] / 2)
    if var_x:
      X, p = Util.parameter(X_RANGE[0], X_RANGE[1])
      parameters *= p

    Y = Y_RANGE[0]
    if var_y:
      
      Y, p = Util.parameter(0, SIZE[0]-MAX_LENGTH)
      # print 'Y',Y
      parameters *= p

    WIDTH = 1
    if var_width:
      sizes = [1, 3, 5, 7, 9, 11]
      WIDTH = np.random.choice(sizes)
      parameters *= len(sizes)

    sparse = [Y, X, LENGTH, WIDTH]

    image = np.zeros(SIZE, dtype=bool)


    half_width = math.floor(WIDTH / 2) # this always floors
    
    # print(Y,LENGTH,X,half_width,WIDTH)
    image[Y:Y+LENGTH, X-half_width:X+half_width+1] = 1


    label = LENGTH

    return sparse, image, label, parameters

In [6]:
def process_and_save(dataset, output_folder, subset_name):
    # Define image subfolder within output folder
    subset_folder = os.path.join(output_folder, subset_name)
    image_subfolder = os.path.join(output_folder, 'images')


    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)


    if not os.path.exists(subset_folder):
        os.makedirs(subset_folder)


    # Initialize list to hold all JSON data
    json_data_list = []


    # Process and save images and labels
    for item in dataset:
        # Load image if it's a URL or a file path
        if isinstance(item['image'], str):
            response = requests.get(item['image'])
            image = Image.open(BytesIO(response.content))
        else:
            image = item['image']  # Assuming it's a PIL.Image object


        # Create a unique ID for each image
        unique_id = str(uuid.uuid4())


        # Define image path
        image_path = os.path.join(image_subfolder, f"{unique_id}.jpg")


        # Save image
        image.save(image_path)


        # Remove duplicates and format answers
        answers = item['answers']
        unique_answers = list(set(answers))
        formatted_answers = ", ".join(unique_answers)


        # Structure for LLaVA JSON
        json_data = {
            "id": unique_id,
            "image": f"{unique_id}.jpg",
            "conversations": [
                {
                    "from": "human",
                    "value": item['question']
                },
                {
                    "from": "gpt",
                    "value": formatted_answers
                }
            ]
        }


        # Append to list
        json_data_list.append(json_data)


    # Save the JSON data list to a file
    json_output_path = os.path.join(output_folder, subset_name, 'dataset.json')
    with open(json_output_path, 'w') as json_file:
        json.dump(json_data_list, json_file, indent=4)


def save_dataset(dataset_name, output_folder, class_name, subset_name, val_samples=None):
    # Load the dataset from Hugging Face
    dataset = load_dataset(dataset_name, split=subset_name)


    # Filter for images with the specified class in 'question_type'
    filtered_dataset = [item for item in dataset if item['question_type'] == class_name]


    # Determine the split for training and validation
    if val_samples is not None and subset_name == 'train':
        train_dataset = filtered_dataset[val_samples:]
        val_dataset = filtered_dataset[:val_samples]
    else:
        train_dataset = filtered_dataset
        val_dataset = []


    # Process and save the datasets
    for subset, data in [('train', train_dataset), ('validation', val_dataset)]:
        if data:
            process_and_save(data, output_folder, subset)




In [ ]:
output_folder = 'dataset'
class_name = 'other'
val_samples = 300
save_dataset('Multimodal-Fatima/OK-VQA_train', output_folder, class_name, 'train', val_samples)
save_dataset('Multimodal-Fatima/OK-VQA_test', output_folder, class_name, 'test')


In [20]:
import os


In [23]:
def generate_save_dataset(output_folder, subset_name, number_of_data):
    
    subset_folder = os.path.join(output_folder, subset_name)
    image_subfolder = os.path.join(output_folder, 'images')


    if not os.path.exists(image_subfolder):
        os.makedirs(image_subfolder)


    if not os.path.exists(subset_folder):
        os.makedirs(subset_folder)


    # Initialize list to hold all JSON data
    json_data_list = []

    for i in range(number_of_data):
        sparse, image, answer, parameters = length()
        int_array = image.astype(int)
        uint8_array = (int_array * 255).astype(np.uint8) 
        
        img = Image.fromarray(uint8_array)
        
        unique_id = str(uuid.uuid4())


        # Define image path
        image_path = os.path.join(image_subfolder, f"{unique_id}.jpg")


        # Save image
        img.save(image_path)

        json_data = {
            "id": unique_id,
            "image": f"{unique_id}.jpg",
            "conversations": [
                {
                    "from": "human",
                    "value": 'What is the length of the line in this picture?'
                },
                {
                    "from": "gpt",
                    "value": str(answer)
                }
            ]
        }


        # Append to list
        json_data_list.append(json_data)

        json_output_path = os.path.join(output_folder, subset_name, 'dataset.json')
        with open(json_output_path, 'w') as json_file:
            json.dump(json_data_list, json_file, indent=4)

    

In [24]:
output_folder = 'lengthDataset'
number_of_data = 300

generate_save_dataset(output_folder, 'train', number_of_data)
generate_save_dataset(output_folder, 'validation', number_of_data)




In [32]:
generate_save_dataset(output_folder, 'test', 10)

In [7]:
from llava.model.builder import load_pretrained_model
from llava.mm_utils import get_model_name_from_path
from llava.eval.run_llava import eval_model

model_path = "liuhaotian/llava-v1.5-7b"

tokenizer, model, image_processor, context_len = load_pretrained_model(
    model_path=model_path,
    model_base=None,
    model_name=get_model_name_from_path(model_path),
    offload_folder="/content/llava_model"
)

[2024-03-07 18:27:34,391] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)


You are using a model of type llava to instantiate a model of type llava_llama. This is not supported for all configurations of models and can yield errors.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [9]:
os.chdir("LLaVA")


In [17]:
import wandb
wandb.login(key="*******************************")
wandb.init()


wandb: Currently logged in as: mahsa-geshvadi001 (mpsych). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /home/mahsa.geshvadi001/.netrc


In [25]:
# Assign paths to variables
DEEPSPEED_SCRIPT = "deepspeed llava/train/train_mem.py"
DEEPSPEED_JSON = "./scripts/zero3.json"
MODEL_NAME = "liuhaotian/llava-v1.5-7b"
DATA_PATH = "lengthDataset/train/dataset.json"  # Replace with your JSON data path
IMAGE_FOLDER = "lengthDataset/images"  # Replace with your image folder path
VISION_TOWER = "openai/clip-vit-large-patch14-336"
OUTPUT_DIR = "lengthDataset/"  

In [26]:
# Command to run the script
finetune_script = f'''
{DEEPSPEED_SCRIPT} \
    --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 \
    --deepspeed {DEEPSPEED_JSON} \
    --model_name_or_path {MODEL_NAME} \
    --version v1 \
    --data_path {DATA_PATH} \
    --image_folder {IMAGE_FOLDER} \
    --vision_tower {VISION_TOWER} \
    --mm_projector_type mlp2x_gelu \
    --mm_vision_select_layer -2 \
    --mm_use_im_start_end False \
    --mm_use_im_patch_token False \
    --image_aspect_ratio pad \
    --group_by_modality_length True \
    --bf16 True \
    --output_dir {OUTPUT_DIR} \
    --num_train_epochs 5 \
    --per_device_train_batch_size 16 \
    --per_device_eval_batch_size 4 \
    --gradient_accumulation_steps 1 \
    --evaluation_strategy "no" \
    --save_strategy "steps" \
    --save_steps 50000 \
    --save_total_limit 1 \
    --learning_rate 2e-4 \
    --weight_decay 0. \
    --warmup_ratio 0.03 \
    --lr_scheduler_type "cosine" \
    --logging_steps 1 \
    --tf32 True \
    --model_max_length 2048 \
    --gradient_checkpointing True \
    --dataloader_num_workers 4 \
    --lazy_preprocess True \
    --report_to wandb
'''

In [27]:
import torch
torch.cuda.empty_cache()

In [28]:
!{finetune_script}

[2024-03-07 19:04:41,503] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
[2024-03-07 19:04:44,182] [WARNING] [runner.py:202:fetch_hostfile] Unable to find hostfile, will proceed with training with local resources only.
Detected CUDA_VISIBLE_DEVICES=0: setting --include=localhost:0
[2024-03-07 19:04:44,183] [INFO] [runner.py:568:main] cmd = /home/mahsa.geshvadi001/miniconda3/envs/LLMP/bin/python -u -m deepspeed.launcher.launch --world_info=eyJsb2NhbGhvc3QiOiBbMF19 --master_addr=127.0.0.1 --master_port=29500 --enable_each_rank_log=None llava/train/train_mem.py --lora_enable True --lora_r 128 --lora_alpha 256 --mm_projector_lr 2e-5 --deepspeed ./scripts/zero3.json --model_name_or_path liuhaotian/llava-v1.5-7b --version v1 --data_path lengthDataset/train/dataset.json --image_folder lengthDataset/images --vision_tower openai/clip-vit-large-patch14-336 --mm_projector_type mlp2x_gelu --mm_vision_select_layer -2 --mm_use_im_start_end False --mm_us

In [33]:
!python scripts/merge_lora_weights.py --model-path lenghtDataset/checkpoints --model-base liuhaotian/llava-v1.5-7b --save-model-path lengthDataset/




[2024-03-07 19:38:00,754] [INFO] [real_accelerator.py:191:get_accelerator] Setting ds_accelerator to cuda (auto detect)
Traceback (most recent call last):
  File "/home/mahsa.geshvadi001/Projects/LLaVA/scripts/merge_lora_weights.py", line 22, in <module>
    merge_lora(args)
  File "/home/mahsa.geshvadi001/Projects/LLaVA/scripts/merge_lora_weights.py", line 8, in merge_lora
    tokenizer, model, image_processor, context_len = load_pretrained_model(args.model_path, args.model_base, model_name, device_map='cpu')
                                                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mahsa.geshvadi001/Projects/LLMP_mahsa/LLMP/LLMP/LLaVA/llava/model/builder.py", line 121, in load_pretrained_model
    model = AutoModelForCausalLM.from_pretrained(model_base, low_cpu_mem_usage=True, **kwargs)
            ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/home/mahsa.geshvad